In [1]:
import numpy as np
import torch

from torch.utils.data import Dataset

import torch.nn.functional as F
import torch.nn as nn

import matplotlib.pyplot as plt

import myModelinTF

from torch.optim.lr_scheduler import StepLR
import pandas as pd
import tensorflow as tf
from torch.autograd import Variable 

from statistics import mean 
import random

In [2]:
demo = np.load('dataset_train_firsts.npy', allow_pickle=True)
print("Train firsts set size: " + str(len(demo)))

Train firsts set size: 495


In [3]:
# def permute_them(x):
#   return torch.permute(x, (1, 0, 2))

In [4]:
def fill_four_with_last(arr):
    prev = np.arange(len(arr))
    prev[arr == 4] = 0
    prev = np.maximum.accumulate(prev)

    # it can be that the first element is 4, so remove those
    temp = arr[prev]
    new_deleted = np.delete(temp, np.where(temp == 4))
    count = len(temp) - len(new_deleted)
    new = np.insert(new_deleted, 0, np.repeat(new_deleted[0], count), axis=0)
    return new

In [5]:
class PG(nn.Module):
    def __init__(self, n_actions):
        super().__init__()
        self.n_actions = n_actions

        # Policy takes input initial conditions and outputs planning route, open-loop control
        self.actor_model = nn.Sequential(
            nn.LSTM(input_size = 1, hidden_size = self.n_actions, num_layers = 2, dropout = 0.1,batch_first = True),
        )

        self.logprobs = []
        self.state_values = []
        self.rewards = []

    def forward(self, states):
        logits, (h_T, c_T)  = self.actor_model(states)
        return logits[-1]
    
    # Run agent in environment to create sample trajectories by generator
    # The environment model is a seq2seq model
    def rollout_policy(self, sysmodel, policy, init, traj_idx, t_step = 20, t_max=200):

        states, actions, actions_probs_policy, rewards = [], [], [], []
        init_idx = init[traj_idx]
        states_init_idx = init_idx[0]
        init_COT_idx = states_init_idx[1,0:t_step]

        states = torch.FloatTensor(init_COT_idx.T)

        states_history = states.unsqueeze(0)
        #states_history = states[-1,:].unsqueeze(0)
        list_actions_taken = []
        actions_probs_policy_taken = []
        action = 0
        for t in range(0, t_max, t_step):
            logits = policy(states_history) # forward pass
            
            probs = nn.functional.softmax(logits, -1) # get estimated actions for states

            actions_probs_policy = np.squeeze(probs).clone().detach().numpy()
            actions_probs_policy_rep = np.tile(actions_probs_policy,(t_step,1))
            actions_probs_policy_taken.append(actions_probs_policy_rep)

            action_prev = action
            action = np.random.choice(3,  p = actions_probs_policy)
            if action == 2:
                action = action_prev

            action_rep = np.repeat(action, t_step)
            list_actions_taken.append(action_rep)
            
            #list_actions_filled = np.append(list_actions_filled, a)
            
            # Apply dict to go from 1,2,3,4 as action to [0,2400,0] [0,2400,1000]  [2300,0,0]  [2300,0,1000] 
            my_dict = {0:[0,2400,1000], 1:[2300,0,1000], 2: [2300,0,0]} 
            list_actions_taken_np = np.array(list_actions_taken) 
            list_actions_taken_np = np.append([2, 2, 2, 2, 2], list_actions_taken_np)
            actions_array_mapped = map(my_dict.get, list_actions_taken_np.flatten())
            actions_array_mapped = np.array(list(actions_array_mapped))
            
            actions_array_mapped = (actions_array_mapped - norm_params_actions[:,0]) / norm_params_actions[:,1]

            actions_sysmodel = np.concatenate([np.arange(actions_array_mapped.shape[0])[:,None]+1,actions_array_mapped], axis=1) #maybe needed [:,None]
            actions_sysmodel = tf.expand_dims(actions_sysmodel, axis = 0, name=None)
            
            states = sysmodel(actions_sysmodel)

            # Revert normalized values for states

            states = states*norm_params[:,1]+norm_params[:,0]
            states_COT_np = np.array(states[:,:,0])
            states = torch.FloatTensor(states_COT_np[:,:-1])
            # More actions fed to the sysmodel, higher the accuracy, but the resulting states will not be the ones effectively experienced, so history is needed
            # Rewards can be calculated on the set of states experienced with all actions at once, but then the scope is just to evaluate the set of actions in the whole episode, not step by step
            states_history = torch.cat((states, states[-t_step:]), 1)

            # if done:
            #     break
        
        # print(actions_taken)
        # actions_taken = np.array(actions_taken)
        actions_probs_policy_taken = np.array(actions_probs_policy_taken).reshape(t_max, 3)
        # states_all = sysmodel(actions_taken_reshape)
        # states_all = states_new*norm_params[:,0]+norm_params[:,1]
        rewards = get_rewards(states, np.array(list_actions_taken), t_max)
        return states_history, states, list_actions_taken_np, actions_probs_policy_taken, rewards
    
    def train(self, sysmodel, policy, critic, init, traj_idx, t_step = 20, t_max=200, gamma = 0.9):
        #policy.train()

        states, actions, actions_probs_policy, rewards, list_actions_taken, actions_probs_policy_taken = [], [], [], [], [], []
        init_idx = init[traj_idx]
        states_init_idx = init_idx[0]
        actions_init_idx = init_idx[1]
        init_COT_idx = states_init_idx[1,0:t_step]

        states = torch.FloatTensor(init_COT_idx.T)

        states_history = states.unsqueeze(0)

        action = 0

        step = 1
        for t in range(0, t_max, t_step):
            
            # Forward pass policy
            logits = policy(states_history) 
            
            # Get estimated actions for states in scale [0,1]
            probs = nn.functional.softmax(logits, -1) 

            # Keep action probs for t_step - Repeated action concept
            actions_probs_policy = np.squeeze(probs).clone().detach().numpy()
            actions_probs_policy_rep = np.tile(actions_probs_policy,(t_step,1))
            actions_probs_policy_taken.append(actions_probs_policy_rep)

            # Action 2 has the meaning "Keep previous action"
            # The other two actions : 0:[0,2400,1000], 1:[2300,0,1000]
            action_prev = action
            action = np.random.choice(3,  p = actions_probs_policy)
            if action == 2:
                action = action_prev

            # Keep action for t_step too - Repeated action concept
            action_rep = np.repeat(action, t_step)
            list_actions_taken.append(action_rep)
            
            # Apply dict to go from 1,2,3,4 as action to [0,2400,0] [0,2400,1000]  [2300,0,0]  [2300,0,1000] 
            my_dict = {0:[0,2400,1000], 1:[2300,0,1000], 2: [2300,0,0]} 
            list_actions_taken_np = np.array(list_actions_taken) 
            list_actions_taken_np = np.append([2, 2, 2, 2, 2], list_actions_taken_np)
            list_actions_taken_np = list_actions_taken_np.flatten()
            actions_array_mapped = map(my_dict.get, list_actions_taken_np)
            actions_array_mapped = np.array(list(actions_array_mapped))
            
            # Normalize actions wrt to also states, mean, stdev calculated on offline CFD data
            actions_array_mapped = (actions_array_mapped - norm_params_actions[:,0]) / norm_params_actions[:,1]

            # The input to the sysmodel is also an absolute time index
            actions_sysmodel = np.concatenate([np.arange(actions_array_mapped.shape[0])[:,None]+1,actions_array_mapped], axis=1) 
            actions_sysmodel = tf.expand_dims(actions_sysmodel, axis = 0, name=None)
            
            # Take a step in the environment **with all previous actions taken**
            states = sysmodel(actions_sysmodel)

            # Revert normalization step for states
            states = states*norm_params[:,1]+norm_params[:,0]

            # From output consider only COT value
            states_COT_np = np.array(states[:,:,0])
            states = torch.FloatTensor(states_COT_np[:,:-1])
            
            # States is the output from sysmodel, 
            # states_history is incrementally saving states at every t_step 
            states_history = torch.cat((states_history, states[-t_step:]), 1)

            # Flatten 
            actions_probs_policy_taken_np = np.array(actions_probs_policy_taken).reshape(step*t_step, 3)

            # Calculate Loss Actor and Critic

            # Skip first iter, since there is no "previous value"
            if t > 4*t_step:

                # When the scope is to get an action sequence only, use states for get_rewards()
                rewards = get_rewards(states[:, -t_step*2:-t_step], np.array(list_actions_taken), t_max)

                # Actor
                states_np = states.numpy()

                input1 = states_np[:,-1]  
                for i in range(4):
                    input1 = np.concatenate((input1, [np.max(states_np[:,-5*(i+1):-5*(i)-1])]))
                v_prime = critic(input1)

                input2 = states_np[:,-t_step]
                for i in range(4):
                    k = i+6
                    input2 = np.concatenate((input2, [np.max(states_np[:,-5*k:-5*(k-1)])]))
                v = critic(input2)
                
                gamma_r = []
                for i in range(t_step):
                    expt = t_step + i -t_step
                    gamma_r.append(gamma**expt)

                td_error_v = torch.sum(torch.from_numpy(np.array(gamma_r)) * rewards) + v_prime - (gamma**t_step)*v

                actions_tensor = torch.from_numpy(list_actions_taken_np)
                probs_tensor = torch.from_numpy(np.array(actions_probs_policy_taken_np))

                log_probs_for_actions = torch.log(torch.sum(
                    probs_tensor * to_one_hot(actions_tensor[5:], 3), dim =1))
                
                # UPDATING THE POLICY NETWORK
                loss_policy = -(torch.mean(log_probs_for_actions)*td_error_v).sum()

                optimizer_policy.zero_grad()
                #loss_policy.requires_grad = True
                loss_policy.backward()
                torch.nn.utils.clip_grad_norm_(policy.parameters(), 0.01)
                optimizer_policy.step()

                # Critic

                # target = torch.sum(torch.from_numpy(np.array(gamma_r))*rewards) + v_prime
                # predicted = v
                # # UPDATING THE VALUE NETWORK
                # loss_critic = criterionMSE(predicted.to(torch.float32), target.to(torch.float32))/1e3
                
                # optimizer_critic.zero_grad()
                # #loss_critic.requires_grad = True
                # loss_critic.backward()
                # torch.nn.utils.clip_grad_norm_(critic.parameters(), 0.01)
                # optimizer_critic.step()

                #returns = sum(rewards)
                sum_of_loss = loss_policy.item()
                print(f"\nLoss Actor: {sum_of_loss}\n")
                #print(f"\nLoss Critic: {loss_critic.item()}\n")
                

            step += 1

In [6]:
class Critic(nn.Module):
    def __init__(self):
        super().__init__()

        self.critic_model = nn.Sequential(
            nn.Linear(5, 32),
            nn.LeakyReLU(inplace=False),
            nn.Linear(32, 1)
        )

    def forward(self, state):
        state = torch.FloatTensor(state) #.unsqueeze(0)
        state_value = self.critic_model(state)
        return state_value

In [7]:
def ind(array, t_max):
    idx_return = (t_max + 1,0)
    for idx, val in np.ndenumerate(array):
        if val <= 201 and val >= 199:
            idx_return = idx
            break
    return int(idx_return[0])
    # If no item was found return None, other return types might be a problem due to
    # numbas type inference.

In [8]:
def get_rewards(states, actions, t_max):
    
    states = states.numpy()
    states = states.squeeze()
    rewards = np.zeros((np.shape(states)[0],1))

    actions = actions.flatten()

    # print(actions)
    max_hotspot =  np.max(states)
    if max_hotspot < 280:
        rewards[-1] = 1

    target_first_reached_at = ind(states,t_max)

    print(f"Reached Target 200C at step {target_first_reached_at} with t_max = {t_max}")

    if target_first_reached_at < t_max and target_first_reached_at > (t_max - 180):
        rewards[-1] = rewards[-1] + 15

        
    for idx in range(np.shape(states)[0]):
        #idx = idx -1
        # if temps.any() > 230:
        #     rewards[idx] = rewards[idx] -30
        
        # subset = actions[idx-20:]
        # if np.sum(np.diff(subset)) == 0:
        #     rewards[idx] = rewards[idx] + 20

        # subset = states[idx-10:]
        # if np.absolute(np.sum(np.diff(subset))) < 10:
        #     #print('Difference between states not big')
        #     rewards[idx] = rewards[idx] + 70

        # Reduce power consumption
        # if actions[idx] == 1 | actions[idx] == 3:
        #     rewards[idx] = rewards[idx] -10

        # Action 4 is encouraged , has the meaning to keep the previous action
        # if actions[idx] == 2:
        #     #print('Reward for keeping previous action')
        #     rewards[idx] = rewards[idx] + 1

        # Negative temp goes with high negative reward
        if states[idx] < 0:
            #print('Penalizing for negative values')
            rewards[idx] = rewards[idx] -2

        # Penalize if previous state higher than current (variation)
        temps_subset = np.array([states[idx], states[idx-1]])
        if np.diff(temps_subset) <= 0:
            #print('Penalizing for decreasing or steady state value')
            rewards[idx] = rewards[idx] - 5

        if 0 > np.diff(temps_subset) <= 1 :
            #print('Penalizing for decreasing or steady state value')
            rewards[idx] = rewards[idx] + 2

    return rewards

def to_one_hot(y_tensor, ndims):
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    y_one_hot = torch.zeros(
        y_tensor.size()[0], ndims).scatter_(1, y_tensor, 1)
    return y_one_hot

In [9]:
def sample_demos(demos, traj_idx):
    demo = demos[traj_idx]
    states_demo = demo[0]
    
    COT_demo = states_demo[1]
    states = torch.FloatTensor(COT_demo.T)
    states = states.unsqueeze(0)

    list_actions_taken = np.array(demo[1])
    actions_probs_policy_taken = demo[2]

    rewards = get_rewards(states, list_actions_taken, 480)
    
    return states, list_actions_taken, actions_probs_policy_taken, rewards

In [ ]:
# DEVICE
DEVICE = 'cpu'

# ENV SETUP
sysmodel = myModelinTF.load_model()
n_actions = 3 # excluded 1:[0,2400,1000], 3:[2300,0,1000], then returned now excluded {0:[0,2400,0], 1:[2300,0,0]}

# INITILIZING POLICY AND REWARD FUNCTION
policy = PG(n_actions)
policy.to(DEVICE)

critic = Critic()
critic.to(DEVICE)

norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T
norm_params_actions = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params_actions.csv', header = None).to_numpy().T


# LOADING EXPERT/DEMO SAMPLES
DATASETNPY = "dataset_train_firsts.npy"

DATASET_DEMO = "dataset_train.npy"

# Mean stdev from CFD sims
norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T
norm_params_actions = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params_actions.csv', header = None).to_numpy().T

##### TUNE MODEL WITH REWARD LOSS

mean_rewards = []
mean_loss = []
mean_loss_rew = []

# EACH BATCH HAS TRAJECTORIES
EPISODES_FROM_DEMO = 3
EPISODES_TO_PLAY = 5

EPISODES_TO_TEST = 5

BATCHES_DEMO = 200
#ITERS_POLICY = 3


# Retrain all params
optimizer_policy = torch.optim.Adam(policy.parameters(), 1e-1, weight_decay=1e-2)
optimizer_critic = torch.optim.Adam(critic.parameters(), 1e-1, weight_decay=1e-2)


init_states = np.load(DATASETNPY, allow_pickle=True)
demos = np.load(DATASET_DEMO, allow_pickle=True)

criterionMSE = nn.MSELoss()

t_step = 10
t_max = 500

gamma = 0.4
############## OFFLINE RL - PRETRAIN CRITIC ####################
loss_critic_lst = []
for i in range(BATCHES_DEMO):
    trajs_demo = [sample_demos(demos, l) for l in random.sample(range(400), EPISODES_FROM_DEMO)] 
    expected_rewards_lst = []
    rewards_to_go_lst = []
    for traj in trajs_demo:
        states, actions, _, rewards = traj
        step = 1
        for t in range(0, t_max, t_step):
            states_batch = states[:, 0:(step+1)*t_step]
            actions_batch = actions[0:(step+1)*t_step]

             # Skip first iter, since there is no "previous value"
            if t > 4*t_step:

                # When the scope is to get an action sequence only, use states for get_rewards()
                rewards = get_rewards(states_batch[:, -t_step*2:-t_step], np.array(actions_batch), t_max)

                # For Critic Update - TD ERROR
                states_batch_np = states_batch.numpy()

                input1 = states_batch_np[:,-1]  
                for i in range(4):
                    input1 = np.concatenate((input1, [np.max(states_batch_np[:,-5*(i+1):-5*(i)-1])]))
                v_prime = critic(input1)

                input2 = states_batch_np[:,-t_step]
                for i in range(4):
                    k = i+6
                    input2 = np.concatenate((input2, [np.max(states_batch_np[:,-5*k:-5*(k-1)])]))
                v = critic(input2)

                # Critic

                gamma_r = []
                for i in range(t_step):
                    expt = t_step + i -t_step
                    gamma_r.append(gamma**expt)

                target = torch.sum(torch.from_numpy(np.array(gamma_r))*rewards) + v_prime
                predicted = v
                # UPDATING THE VALUE NETWORK
                loss_critic = criterionMSE(predicted.to(torch.float32), target.to(torch.float32))/1e3
                
                #optimizer_critic.zero_grad()
                loss_critic.backward()
                torch.nn.utils.clip_grad_norm_(critic.parameters(), 0.01)
                optimizer_critic.step()

                print(f"\nLoss Critic: {loss_critic.item()}\n")
                loss_critic_lst.append(loss_critic.item())

            step += 1

plt.subplot(1, 1, 1)
plt.title(f"Critic Pre-train Loss")
plt.plot(loss_critic_lst)
plt.grid()

plt.savefig('Critic Pretrain Loss.png')
plt.close()

############## ON-POLICY #####################
rewards_sum_plot = np.array((0))
log_probs_for_actions_lst, cumulative_returns_lst = [], []
#for i in range(ITERS_POLICY):
[policy.train(sysmodel, policy, critic, init_states, l, t_step, t_max) for l in random.sample(range(400), EPISODES_TO_PLAY)]
  
trajs_test = [policy.rollout_policy(sysmodel, init_states, l, t_step, t_max) for l in random.sample(range(400), EPISODES_TO_TEST)]
for traj in trajs_test:
    states_history, states, actions, probs, rewards = traj

    rewards_sum_plot = np.column_stack((rewards_sum_plot, np.array(rewards.sum())))

    states_history_plot = np.array(states_history.squeeze())
    states_plot = np.array(states.squeeze())
    rewards_sum_plot2 = rewards_sum_plot.squeeze()

    plt.subplot(2, 2, 1)
    plt.title(f"Rewards Sum")
    plt.plot(rewards_sum_plot2[1:])
    plt.grid()

    plt.subplot(2, 2, 2)
    plt.title(f"Activations")
    plt.plot(actions[0:500])
    plt.grid()

    plt.subplot(2, 2, 3)
    plt.title(f"Temps Actual")
    plt.plot(states_history_plot[0:500].squeeze(), color='red')
    plt.grid()

    plt.subplot(2, 2, 4)
    plt.title(f"Temps Planned")
    plt.plot(states_plot[0:500].squeeze(), color = 'm')
    plt.grid()

    plt.savefig('Actor Critic Testing.png')
    plt.close()